In [1]:
# Validation of the steady-state simulation - Case 3: AeroCNN-II
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import datetime
%matplotlib inline

In [2]:
### Defining parameters and hyperparameters of the model
n_layers=4
n_kernels=300 # Number of kernels in convolutional network
kernel_size1=2
strides=1
n_units=128 # Number of units in the hidden layer of the MLP network
input_size = 100 # Size of input for the network (100 coefficients and 3 other parameters, AoA, h, beta)
lr = 1e-04 # Learning rate of the network
test_rate=0.1 # Defines the ratio of training dataset and test dataset
val_rate=0.2
n_data = 16 # Number of txt files from which the aerodynamic coefficients are extracted
batch_size = 20 # Mini-batch size

In [3]:
# Changing working directory

main_directory = 'D:\\VAWT_data\\flap_steady\\flap_steady'
os.chdir(main_directory)

In [4]:
## Basic parameters

c = 1 # Chord length
h = np.array([0.01, 0.02, 0.03]) * c # Height of the Gurney flaps
t = 0.02 * h # Thickness of the Gurney flaps
alpha = np.linspace(0, 16, 9).reshape((9,1)) # Angles of attack
beta = np.linspace(30, 90, 5).reshape((5,1))

h = h.reshape((-1,1))
t = t.reshape((-1,1))

In [5]:
n_alpha = alpha.shape[0] # Number of the angles of attack
n_beta = beta.shape[0] # Number of the Gurney flap inclination
n_h = h.shape[0] # Number of the height of the Gurney flaps
n_cases = n_data * n_alpha
aa_ = np.zeros((n_cases,1))
for i in range(0, n_data):
    aa_[n_alpha*i:n_alpha*(i+1),:] = alpha[:,:]
    
aa = aa_ / np.max(alpha)

In [6]:
h = h.reshape((-1,1))
t = t.reshape((-1,1))

In [7]:
n_alpha = alpha.shape[0] # Number of the angles of attack
n_beta = beta.shape[0] # Number of the Gurney flap inclination
n_h = h.shape[0] # Number of the height of the Gurney flaps
n_cases = n_data * n_alpha # Total number of cases(Number of geometries * Number of angles of attack)

In [8]:
## Creating input dataset - mask
mask_dir = main_directory + '\\aeroCNN2Mask'
os.chdir(mask_dir)
mask_list = os.listdir(mask_dir)
mask_target = [file for file in mask_list if file.endswith('.csv')]

In [9]:
mask_df = pd.DataFrame()
for file in mask_target:
    data = pd.read_csv(file, header=None)
    mask_df = pd.concat([mask_df, data], axis=0)

In [10]:
x = mask_df.iloc[:,:].values

In [11]:
x_alligned = np.zeros((x.shape[0],x.shape[1]))
for i in range(0,144*2):
    if i%2==0:
        x_alligned[i*100:(i+1)*100,:] = x[i*100:(i+1)*100,:]
    else:
        x_alligned[i*100:(i+1)*100,:] = x[i*100:(i+1)*100,:].T


In [12]:
x = x_alligned.reshape((16*9,2,input_size,input_size))

In [13]:
## Creating output dataset - Cl and Cd
files_orig = os.listdir(main_directory)
files_target = [file for file in files_orig if file.endswith('.txt')]
os.chdir(main_directory)

In [14]:
target_df = pd.DataFrame()
for file in files_target:
    data = pd.read_table(file, header=None)
    target_df = pd.concat([target_df, data], axis=0)

In [15]:
cd_orig = target_df.iloc[:,3].values # Cd values
#target_c4 = target_df.iloc[:,4].values# Cl values

In [16]:
cd = (cd_orig-np.min(cd_orig))/(np.max(cd_orig)-np.min(cd_orig))

In [17]:
y = cd.reshape((-1,1))

In [18]:
x_all, x_test, y_all, y_test = train_test_split(x, y, test_size=test_rate, random_state=1)

In [19]:
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=val_rate/(1-test_rate), random_state=1)

In [20]:
x_train.shape

(100, 2, 100, 100)

In [21]:
y_train.shape

(100, 1)

In [22]:
tf.keras.backend.set_image_data_format('channels_first')

In [23]:
input_image = tf.keras.Input(shape=(2, input_size, input_size))

x_conv1 = tf.keras.layers.Conv2D(n_kernels, (kernel_size1, kernel_size1), strides=(strides, strides),
                                activation='relu', padding='same'
                                )(input_image)
x1_ = tf.keras.layers.MaxPooling2D((2,2))(x_conv1)
x_conv2 = tf.keras.layers.Conv2D(n_kernels, (kernel_size1, kernel_size1), strides=(strides, strides),
                                activation='relu', padding='same'
                                )(x1_)
x2_ = tf.keras.layers.MaxPooling2D((2,2))(x_conv2)
x_conv3 = tf.keras.layers.Conv2D(n_kernels, (kernel_size1, kernel_size1), strides=(strides, strides),
                                activation='relu', padding='same'
                                )(x2_)
x3_ = tf.keras.layers.MaxPooling2D((2,2))(x_conv3)
x_conv4 = tf.keras.layers.Conv2D(n_kernels, (kernel_size1, kernel_size1), strides=(strides, strides),
                                activation='relu', padding='same'
                                )(x3_)
x4_ = tf.keras.layers.MaxPooling2D((2,2))(x_conv4)
x_flat = tf.keras.layers.Flatten()(x4_)
x_fc1 = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc1')(x_flat)
x_fc2 = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc2')(x_fc1)
x_fc3 = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc3')(x_fc2)
x_fc4 = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc4')(x_fc3)
x_fc5 = tf.keras.layers.Dense(units=n_units, activation='relu', name='fc5')(x_fc4)

output_data = tf.keras.layers.Dense(units=1, activation='linear', name='outputLayer')(x_fc5)
# AeroCNN-II based
model = tf.keras.Model(input_image, output_data)

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 100, 100)]     0         
                                                                 
 conv2d (Conv2D)             (None, 300, 100, 100)     2700      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 300, 50, 50)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 300, 50, 50)       360300    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 300, 25, 25)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 300, 25, 25)       360300

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")])

In [26]:
model_directory = "D:\\TrainedModels\\20221130"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

In [27]:
# ckpt_name = model_directory + "\\20221130steadyValidation_AeroCNN2_val_"+str(val_rate) + "_test"+str(test_rate)+ "_" + str(n_units) +"units_OptimalSettings_Cdcheckpoint.h5"

# ckpt = tf.keras.callbacks.ModelCheckpoint(ckpt_name, monitor="val_loss", mode='min', verbose=1, save_best_only=True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, min_delta=6e-7,
                                      restore_best_weights=True, verbose=1)
rp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=200, factor=0.5,
                                          min_delta = 1e-09, min_lr=1e-06, verbose=1)

In [28]:
STEP_SIZE_TRAIN = len(x_train)//batch_size
VALIDATION_STEPS = len(x_val)//batch_size

In [29]:
start = datetime.datetime.now()
history = model.fit(x_train, y_train, batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    steps_per_epoch = STEP_SIZE_TRAIN, validation_steps=VALIDATION_STEPS,
                    epochs=10000, shuffle=True, callbacks=[es, rp])
end = datetime.datetime.now()

Epoch 1/10000
5/5 [==============================] - 7s 75ms/step - loss: 0.0753 - rmse: 0.2745 - val_loss: 0.0090 - val_rmse: 0.0949 - lr: 1.0000e-04
Epoch 2/10000
5/5 [==============================] - 0s 30ms/step - loss: 0.0116 - rmse: 0.1077 - val_loss: 0.0029 - val_rmse: 0.0535 - lr: 1.0000e-04
Epoch 3/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.0071 - rmse: 0.0843 - val_loss: 0.0046 - val_rmse: 0.0682 - lr: 1.0000e-04
Epoch 4/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.0046 - rmse: 0.0676 - val_loss: 0.0081 - val_rmse: 0.0898 - lr: 1.0000e-04
Epoch 5/10000
5/5 [==============================] - 0s 29ms/step - loss: 0.0056 - rmse: 0.0747 - val_loss: 0.0043 - val_rmse: 0.0654 - lr: 1.0000e-04
Epoch 6/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.0075 - rmse: 0.0863 - val_loss: 0.0104 - val_rmse: 0.1021 - lr: 1.0000e-04
Epoch 7/10000
5/5 [==============================] - 0s 28ms/step - loss: 0.0054 - rmse: 0.073

5/5 [==============================] - 0s 27ms/step - loss: 2.2951e-04 - rmse: 0.0151 - val_loss: 0.0026 - val_rmse: 0.0512 - lr: 1.0000e-04
Epoch 55/10000
5/5 [==============================] - 0s 27ms/step - loss: 2.1596e-04 - rmse: 0.0147 - val_loss: 0.0028 - val_rmse: 0.0526 - lr: 1.0000e-04
Epoch 56/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.2339e-04 - rmse: 0.0111 - val_loss: 0.0028 - val_rmse: 0.0533 - lr: 1.0000e-04
Epoch 57/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.8845e-04 - rmse: 0.0137 - val_loss: 0.0026 - val_rmse: 0.0512 - lr: 1.0000e-04
Epoch 58/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.3087e-04 - rmse: 0.0114 - val_loss: 0.0027 - val_rmse: 0.0517 - lr: 1.0000e-04
Epoch 59/10000
5/5 [==============================] - 0s 28ms/step - loss: 1.2929e-04 - rmse: 0.0114 - val_loss: 0.0031 - val_rmse: 0.0552 - lr: 1.0000e-04
Epoch 60/10000
5/5 [==============================] - 0s 27ms/step - loss: 9.58

5/5 [==============================] - 0s 27ms/step - loss: 2.2010e-04 - rmse: 0.0148 - val_loss: 0.0031 - val_rmse: 0.0555 - lr: 1.0000e-04
Epoch 107/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.7511e-04 - rmse: 0.0132 - val_loss: 0.0029 - val_rmse: 0.0539 - lr: 1.0000e-04
Epoch 108/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.5066e-04 - rmse: 0.0158 - val_loss: 0.0032 - val_rmse: 0.0564 - lr: 1.0000e-04
Epoch 109/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.5834e-04 - rmse: 0.0161 - val_loss: 0.0027 - val_rmse: 0.0516 - lr: 1.0000e-04
Epoch 110/10000
5/5 [==============================] - 0s 27ms/step - loss: 2.6718e-04 - rmse: 0.0163 - val_loss: 0.0029 - val_rmse: 0.0541 - lr: 1.0000e-04
Epoch 111/10000
5/5 [==============================] - 0s 26ms/step - loss: 3.0429e-04 - rmse: 0.0174 - val_loss: 0.0028 - val_rmse: 0.0526 - lr: 1.0000e-04
Epoch 112/10000
5/5 [==============================] - 0s 26ms/step - loss

5/5 [==============================] - 0s 26ms/step - loss: 1.3615e-04 - rmse: 0.0117 - val_loss: 0.0028 - val_rmse: 0.0532 - lr: 1.0000e-04
Epoch 159/10000
5/5 [==============================] - 0s 26ms/step - loss: 1.6049e-04 - rmse: 0.0127 - val_loss: 0.0026 - val_rmse: 0.0507 - lr: 1.0000e-04
Epoch 160/10000
5/5 [==============================] - 0s 26ms/step - loss: 1.1388e-04 - rmse: 0.0107 - val_loss: 0.0025 - val_rmse: 0.0499 - lr: 1.0000e-04
Epoch 161/10000
5/5 [==============================] - 0s 26ms/step - loss: 1.7494e-04 - rmse: 0.0132 - val_loss: 0.0025 - val_rmse: 0.0503 - lr: 1.0000e-04
Epoch 162/10000
5/5 [==============================] - 0s 26ms/step - loss: 7.6076e-05 - rmse: 0.0087 - val_loss: 0.0030 - val_rmse: 0.0546 - lr: 1.0000e-04
Epoch 163/10000
5/5 [==============================] - 0s 26ms/step - loss: 1.2862e-04 - rmse: 0.0113 - val_loss: 0.0025 - val_rmse: 0.0505 - lr: 1.0000e-04
Epoch 164/10000
5/5 [==============================] - 0s 26ms/step - loss

5/5 [==============================] - 0s 27ms/step - loss: 1.3220e-05 - rmse: 0.0036 - val_loss: 0.0024 - val_rmse: 0.0491 - lr: 1.0000e-04
Epoch 211/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.0485e-05 - rmse: 0.0032 - val_loss: 0.0026 - val_rmse: 0.0512 - lr: 1.0000e-04
Epoch 212/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.1390e-05 - rmse: 0.0034 - val_loss: 0.0024 - val_rmse: 0.0488 - lr: 1.0000e-04
Epoch 213/10000
5/5 [==============================] - 0s 27ms/step - loss: 9.2073e-06 - rmse: 0.0030 - val_loss: 0.0026 - val_rmse: 0.0513 - lr: 1.0000e-04
Epoch 214/10000
5/5 [==============================] - 0s 27ms/step - loss: 8.3124e-06 - rmse: 0.0029 - val_loss: 0.0024 - val_rmse: 0.0488 - lr: 1.0000e-04
Epoch 215/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.6569e-05 - rmse: 0.0041 - val_loss: 0.0025 - val_rmse: 0.0504 - lr: 1.0000e-04
Epoch 216/10000
5/5 [==============================] - 0s 27ms/step - loss

...

5/5 [==============================] - 0s 27ms/step - loss: 1.4652e-05 - rmse: 0.0038 - val_loss: 0.0025 - val_rmse: 0.0502 - lr: 1.0000e-04
Epoch 315/10000
5/5 [==============================] - 0s 27ms/step - loss: 3.4902e-05 - rmse: 0.0059 - val_loss: 0.0026 - val_rmse: 0.0514 - lr: 1.0000e-04
Epoch 316/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.2428e-05 - rmse: 0.0047 - val_loss: 0.0028 - val_rmse: 0.0533 - lr: 1.0000e-04
Epoch 317/10000
5/5 [==============================] - 0s 26ms/step - loss: 3.0832e-05 - rmse: 0.0056 - val_loss: 0.0026 - val_rmse: 0.0507 - lr: 1.0000e-04
Epoch 318/10000
5/5 [==============================] - 0s 27ms/step - loss: 3.6703e-05 - rmse: 0.0061 - val_loss: 0.0029 - val_rmse: 0.0539 - lr: 1.0000e-04
Epoch 319/10000
5/5 [==============================] - 0s 27ms/step - loss: 2.6816e-05 - rmse: 0.0052 - val_loss: 0.0024 - val_rmse: 0.0490 - lr: 1.0000e-04
Epoch 320/10000
5/5 [==============================] - 0s 27ms/step - loss

5/5 [==============================] - 0s 27ms/step - loss: 5.8005e-05 - rmse: 0.0076 - val_loss: 0.0030 - val_rmse: 0.0552 - lr: 1.0000e-04
Epoch 367/10000
5/5 [==============================] - 0s 27ms/step - loss: 9.7996e-05 - rmse: 0.0099 - val_loss: 0.0027 - val_rmse: 0.0515 - lr: 1.0000e-04
Epoch 368/10000
5/5 [==============================] - 0s 27ms/step - loss: 7.2038e-05 - rmse: 0.0085 - val_loss: 0.0028 - val_rmse: 0.0530 - lr: 1.0000e-04
Epoch 369/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.0059e-04 - rmse: 0.0100 - val_loss: 0.0024 - val_rmse: 0.0495 - lr: 1.0000e-04
Epoch 370/10000
5/5 [==============================] - 0s 26ms/step - loss: 9.0023e-05 - rmse: 0.0095 - val_loss: 0.0029 - val_rmse: 0.0542 - lr: 1.0000e-04
Epoch 371/10000
5/5 [==============================] - 0s 27ms/step - loss: 1.1402e-04 - rmse: 0.0107 - val_loss: 0.0028 - val_rmse: 0.0527 - lr: 1.0000e-04
Epoch 372/10000
5/5 [==============================] - 0s 27ms/step - loss

5/5 [==============================] - 0s 26ms/step - loss: 0.0036 - rmse: 0.0598 - val_loss: 0.0024 - val_rmse: 0.0493 - lr: 1.0000e-04
Epoch 419/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0022 - rmse: 0.0466 - val_loss: 0.0034 - val_rmse: 0.0582 - lr: 1.0000e-04
Epoch 420/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0013 - rmse: 0.0356 - val_loss: 0.0026 - val_rmse: 0.0508 - lr: 1.0000e-04
Epoch 421/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0014 - rmse: 0.0378 - val_loss: 0.0025 - val_rmse: 0.0504 - lr: 1.0000e-04
Epoch 422/10000
5/5 [==============================] - 0s 26ms/step - loss: 9.1361e-04 - rmse: 0.0302 - val_loss: 0.0031 - val_rmse: 0.0561 - lr: 1.0000e-04
Epoch 423/10000
5/5 [==============================] - 0s 26ms/step - loss: 0.0012 - rmse: 0.0350 - val_loss: 0.0037 - val_rmse: 0.0608 - lr: 1.0000e-04
Epoch 424/10000
5/5 [==============================] - 0s 27ms/step - loss: 0.0032 - rmse: 0.0

5/5 [==============================] - 0s 27ms/step - loss: 4.4070e-06 - rmse: 0.0021 - val_loss: 0.0027 - val_rmse: 0.0520 - lr: 5.0000e-05
Epoch 470/10000
5/5 [==============================] - 0s 27ms/step - loss: 3.9265e-06 - rmse: 0.0020 - val_loss: 0.0027 - val_rmse: 0.0518 - lr: 5.0000e-05
Epoch 471/10000
5/5 [==============================] - 0s 26ms/step - loss: 3.2635e-06 - rmse: 0.0018 - val_loss: 0.0027 - val_rmse: 0.0520 - lr: 5.0000e-05
Epoch 472/10000
5/5 [==============================] - 0s 27ms/step - loss: 3.5579e-06 - rmse: 0.0019 - val_loss: 0.0027 - val_rmse: 0.0519 - lr: 5.0000e-05
Epoch 473/10000
5/5 [==============================] - 0s 26ms/step - loss: 3.0528e-06 - rmse: 0.0017 - val_loss: 0.0027 - val_rmse: 0.0516 - lr: 5.0000e-05
Epoch 474/10000
5/5 [==============================] - 0s 26ms/step - loss: 2.2572e-06 - rmse: 0.0015 - val_loss: 0.0027 - val_rmse: 0.0519 - lr: 5.0000e-05
Epoch 475/10000
5/5 [==============================] - 0s 27ms/step - loss

KeyboardInterrupt: 

In [ ]:
time = end - start
print("Training time:", time)

In [ ]:
storage_dir = "D:\\VAWT_data\\flap_steady\\flap_steady\\result\\"+"20221205AeroCNN2_optimalSettings\\test"+str(test_rate)+"Cdonly"
if not os.path.exists(storage_dir):
    os.makedirs(storage_dir)
os.chdir(storage_dir)

In [ ]:
hist = history.history
plt.plot(hist['loss'], lw=2, label='Training loss')
plt.plot(hist['val_loss'], lw=2, label='Validation loss')
plt.title('Training loss (mean squared error)\nAeroCNN-II, optimal settings, $C_d$ only', size=15)
plt.xlabel('Epoch', size=15)
plt.yscale('log')
#plt.ylim([5e-5, 1e-1])
plt.tick_params(axis='both', which='major', labelsize=15)
plt.grid()
plt.legend(loc='upper right')
saveName = "TrainingLoss_test"+str(test_rate) + ".jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
plt.plot(hist['rmse'], lw=2)
plt.plot(hist['val_rmse'], lw=2)
plt.title('Root Mean Squared Error, optimal settings, $C_d$ only', size=15)
plt.xlabel('Epoch', size=15)
plt.tick_params(axis='both', which='major', labelsize=15)
plt.show()

In [ ]:
plt.plot(hist['rmse'], lw=2, label='Training RMSE')
plt.plot(hist['val_loss'], lw=2, label='Validation RMSE')
plt.title('Root Mean Squared Error\nAeroCNN-II, optimal settings, $C_d$ only', size=15)
plt.xlabel('Epoch', size=15)
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=15)
plt.grid()
plt.legend(loc='upper right')
saveName = "RMSE_test"+str(test_rate) + ".jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
train_results = model.evaluate(x_train, y_train)

In [ ]:
test_results = model.evaluate(x_test, y_test)

In [ ]:
decoded_train_ = model.predict(x_train)
decoded_val_ = model.predict(x_val)
decoded_test_ = model.predict(x_test)

In [ ]:
index_train = np.unique(np.where(np.isin(cd, y_train)))
index_val = np.unique(np.where(np.isin(cd, y_val)))
index_test = np.unique(np.where(np.isin(cd, y_test)))

In [ ]:
y_train = y_train*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)
y_val = y_val*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)
y_test = y_test*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)

In [ ]:
decoded_train = decoded_train_*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)
decoded_val = decoded_val_*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)
decoded_test = decoded_test_*(np.max(cd_orig)-np.min(cd_orig))+np.min(cd_orig)

In [ ]:
model_directory = "D:\\TrainedModels\\20221205"
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
os.chdir(model_directory)
model_name = "20221205steadyValidation_AeroCNN2_val_"+str(val_rate)+"_test"+str(test_rate)+ "_" + str(n_kernels) +"units_optimalSettings_Cdonly.h5"
model.save(model_name, overwrite=True, include_optimizer=True, save_format='h5')

In [ ]:
error_train_abs = np.abs(decoded_train - y_train)

In [ ]:
error_test_abs = np.abs(decoded_test - y_test)

In [ ]:
os.chdir(storage_dir)

In [ ]:
l2_error_train = np.sqrt(np.sum((decoded_train - y_train)**2) / np.sum(y_train**2))
print(l2_error_train)

In [ ]:
l2_error_val = np.sqrt(np.sum((decoded_val - y_val)**2) / np.sum(y_val**2))
print(l2_error_val)

In [ ]:
l2_error_test = np.sqrt(np.sum((decoded_test - y_test)**2) / np.sum(y_test**2))
print(l2_error_test)

In [ ]:
l2_error_train_list = []
for i in range(0, len(x_train)):
    l2_error_train_data = np.sqrt(np.sum((decoded_train[i] - y_train[i])**2) / np.sum(y_train[i]**2))
    l2_error_train_list.append(l2_error_train_data)

In [ ]:
l2_error_val_list = []
for i in range(0, len(x_val)):
    l2_error_val_data = np.sqrt(np.sum((decoded_val[i] - y_val[i])**2) / np.sum((y_val[i]+1e-07)**2))
    l2_error_val_list.append(l2_error_val_data)

In [ ]:
l2_error_test_list = []
for i in range(0, len(x_test)):
    l2_error_test_data = np.sqrt(np.sum((decoded_test[i] - y_test[i])**2) / np.sum(y_test[i]**2))
    l2_error_test_list.append(l2_error_test_data)

In [ ]:
plt.plot(np.linspace(1,x_train.shape[0],x_train.shape[0]),
         l2_error_train*np.ones(x_train.shape[0],), 'k', lw=2.5)
plt.scatter(np.linspace(1, x_train.shape[0], x_train.shape[0]), l2_error_train_list, c='b')
plt.xlabel('Index', fontsize=15)
plt.ylabel('$L_2$ error norm', fontsize=15)
plt.yscale('log')
plt.title('$L_2$ error norm distribution - AeroCNN-II, training\nValidation rate {0}, test rate {1}, optimal settings ($C_d$)'.format(
    val_rate, test_rate), fontsize=15
         )
plt.grid()
saveName = "trainingErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
plt.plot(np.linspace(1,x_val.shape[0],x_val.shape[0]),
         l2_error_val*np.ones(x_val.shape[0],), 'k', lw=2.5)
plt.scatter(np.linspace(1, x_val.shape[0], x_val.shape[0]), l2_error_val_list, c='b')
plt.xlabel('Index', fontsize=15)
plt.ylabel('$L_2$ error norm', fontsize=15)
plt.yscale('log')
plt.title('$L_2$ error norm distribution - AeroCNN-II, validation\nValidation rate {0}, test rate {1}, optimal settings ($C_d$)'.format(
    val_rate, test_rate), fontsize=15
         )
plt.grid()
saveName = "validationErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
plt.plot(np.linspace(1,x_test.shape[0],x_test.shape[0]),
         l2_error_test*np.ones(x_test.shape[0],), 'k', lw=2.5)
plt.scatter(np.linspace(1, x_test.shape[0], x_test.shape[0]), l2_error_test_list, c='b')
plt.xlabel('Index', fontsize=15)
plt.ylabel('$L_2$ error norm', fontsize=15)
plt.yscale('log')
plt.title('$L_2$ error norm distribution - AeroCNN-II, test\nValidation rate {0}, test rate {1}, optimal settings ($C_d$)'.format(
    val_rate, test_rate), fontsize=15
         )
plt.grid()
saveName = "testErrorDistribution.jpg"
plt.savefig(saveName, dpi=300, bbox_inches='tight', pad_inches=0.3)
plt.show()

In [ ]:
predicted = model.predict(x)

In [ ]:
def CD_trainTestSplit_Plot(i, CD, cd, aTrain, aTest, iTrain, iTest):
    
    title_0_Cd = 'Gurney flap not attached (NACA0018)\n$C_D$ prediction, L2 error=%.4f' % l2_error_Cd
    #title_0_Cl = 'Gurney flap not attached (NACA0018)\n$C_L$ prediction, L2 error=%.4f' % l2_error_Cl
    
    title_n_Cd = 'Gurney flap attached h=%.2f, '%(h[i]) + r'$\beta$=%d'%(beta[i])+'\n$C_D$ prediction, L2 error=%.4f'%(l2_error_Cd)
    #title_n_Cl = 'Gurney flap attached h=%.2f, '%(h[i]) + r'$\beta$=%d'%(beta[i])+'\n$C_L$ prediction, L2 error=%.4f'%(l2_error_Cl)
    
    if i==0:
#         title_Cd, title_Cl = title_0_Cd, title_0_Cl
#         savename1,savename2 = "CdComparison_NACA0018.jpg", "ClComparison_NACA0018.jpg"
        title_Cd = title_0_Cd
        savename1 = "CdComparison_NACA0018.jpg"
    else:
#         title_Cd, title_Cl = title_n_Cd, title_n_Cl
        title_Cd = title_n_Cd
        savename1 = "CdComparison_h"+str(h[i])+"_beta"+str(beta[i])+".jpg"
#         savename2 = "ClComparison_h"+str(h[i])+"_beta"+str(beta[i])+".jpg"
    
    # CD graph plot
    plt.plot(alpha, CD, 'k-', label='Ground truth')
    plt.plot(alpha, cd, 'k--', label='Predicted value')
    plt.scatter(aTrain, iTrain, color='b', label='Training set')
    plt.scatter(aVal, iVal, color='g', label='Validation set')
    plt.scatter(aTest, iTest, color='r', label='Test set')
    plt.xlabel('AoA (degree)')
    plt.ylabel('$C_D$')
    plt.title(title_Cd, fontsize=15)        
    plt.legend(loc='upper left')
    plt.ylim([0, 0.12])
    plt.grid()
    plt.savefig(savename1, dpi=300, bbox_inches='tight', pad_inches=0.3)
    plt.show()

    '''# CL graph plot
    plt.plot(alpha, CL, 'k-', label='Ground truth')
    plt.plot(alpha, cl, 'k--', label='Predicted value')
    plt.scatter(aTrain, iTrain[:,1]*(np.max(cl_orig)-np.min(cl_orig)) + np.min(cl_orig), color='b', label='Training set')
    plt.scatter(aTest, iTest[:,1]*(np.max(cl_orig)-np.min(cl_orig)) + np.min(cl_orig), color='r', label='Test set')
    plt.xlabel('AoA (degree)')
    plt.ylabel('$C_L$')
    plt.title(title_Cl, fontsize=15)
    plt.legend(loc='upper left')
    plt.ylim([0, 2])
    plt.grid()
    plt.savefig(savename2, dpi=300, bbox_inches='tight', pad_inches=0.3)
    plt.show()'''

In [ ]:
h = [0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.03, 0.03, 0.03, 0.03, 0.03]
beta = [0, 30, 45, 60, 75, 90, 30, 45, 60, 75, 90, 30, 45, 60, 75, 90]
for i in range(0, 16):
    # Index from each dataset
    iTrain = []
    iVal = []
    iTest = []
    
    # Index from input data (alpha, in this case)
    alpha_train = []
    alpha_val = []
    alpha_test = []
    
    predictedValue = model.predict(x[n_alpha*i:n_alpha*(i+1),:])
    y_corres = y[n_alpha*i:n_alpha*(i+1),:]
    
    l2_error_Cd = np.sqrt(np.sum((predictedValue - y_corres)**2) / np.sum(y_corres**2))
    
    if i==0:
        print('NACA0018 airfoil without Gurney flap\nL2 error of Cd: {0:0.4f}'.format(l2_error_Cd))
    else:
        print('L2 error of Cd: {0:0.4f}'.format(l2_error_Cd))
    
    cd = predicted[n_alpha*i:n_alpha*(i+1)]*(np.max(cd_orig)-np.min(cd_orig)) + np.min(cd_orig)
    CD = y[n_alpha*i:n_alpha*(i+1)]*(np.max(cd_orig)-np.min(cd_orig)) + np.min(cd_orig)
    
    for ii, index in enumerate(index_train[np.where((index_train<(i+1)*n_alpha) & ((index_train>=i*n_alpha)))]):
        iTrain.append(predicted[index])
    for kk, index in enumerate(index_val[np.where((index_val<(i+1)*n_alpha) & ((index_val>=i*n_alpha)))]):
        iVal.append(predicted[index])    
    for jj, index in enumerate(index_test[np.where((index_test<(i+1)*n_alpha) & (index_test>=i*n_alpha))]):
        iTest.append(predicted[index])
        
    iTrain = np.array(iTrain)*(np.max(cd_orig)-np.min(cd_orig)) + np.min(cd_orig)
    iTest = np.array(iTest)*(np.max(cd_orig)-np.min(cd_orig)) + np.min(cd_orig)
    iVal = np.array(iVal)*(np.max(cd_orig)-np.min(cd_orig)) + np.min(cd_orig)
    
    for ii, index in enumerate(index_train[np.where((index_train<(i+1)*n_alpha) & ((index_train>=i*n_alpha)))]):
        alpha_train.append(aa[index])
    for kk, index in enumerate(index_val[np.where((index_val<(i+1)*n_alpha) & ((index_val>=i*n_alpha)))]):
        alpha_val.append(aa[index])
    for jj, index in enumerate(index_test[np.where((index_test<(i+1)*n_alpha) & ((index_test>=i*n_alpha)))]):
        alpha_test.append(aa[index])
        
    aTrain = np.array(alpha_train)*np.max(alpha)
    aVal = np.array(alpha_val)*np.max(alpha)
    aTest = np.array(alpha_test)*np.max(alpha)
    
    CD_trainTestSplit_Plot(i, CD, cd, aTrain, aTest, iTrain, iTest)